# Set Membership Coursework

## 1. Implementeation

### Sequential Search 

In [14]:
class SequentialSearch():
    
    def __init__(self):
        # ADD YOUR CODE HERE
        self.items = []
        pass           
        
    def insertElement(self, file):
        inserted = False
        with open(file,'r') as k:
            for strs in k:
                items = strs.split()
                for item in items:
                    self.items.append(item)
        
        return inserted   

    def searchElement(self, file):     
        found = False
        with open(file,'r') as k:
            for elements in k:
                item2 = elements.split()
                for item1 in self.items:
                    if item1 == item2:
                        found = True
                    else:
                        found = False
            if found == False:
                print(found, ',%s is not found' % item2)
        return found   

### Binary Search Tree

### Balanced Search Tree

In [15]:
# Red-Black Search Tree

class RBTree:
    R = True
    B = False
    # R refers to Red; B refers to Black
    
    class Nodes:
        def __init__(self,val,colour):
            self.val = val
            self.colour = colour
            self.left = None
            self.right = None
            
    def __init__(self):
        self.root = None
        
    def searchElement(self,val):
        return self.search(self.root,val)
    
    def search(self,node,val):
        if node is None:
            return False
        else:
            if node.val == val:
                return True
            elif node.val > val:
                return self.search(node.left,val)
            else:
                return self.search(node.right,val)
    
    def insertElement(self, val):
        self.root = self.insert(self.root, val)
        self.root.colour = self.B


    def insert(self, node, val):
        if node is None:
            return self.Nodes(val, self.R)

        if val < node.val:
            node.left = self.insert(node.left, val)
        elif val > node.val:
            node.right = self.insert(node.right, val)
        else:
            # value already exists, do nothing
            return node

        # perform rotations and color flips
        if self.isR(node.right) and not self.isR(node.left):
            node = self.leftRotate(node)
        if self.isR(node.left) and self.isR(node.left.left):
            node = self.rightRotate(node)
        if self.isR(node.left) and self.isR(node.right):
            self.flipColour(node)

        return node

    def isR(self,node):
        if node is None:
            return False
        return node.colour == self.R
  
    def leftRotate(self, node):
        x = node.right
        node.right = x.left
        x.left = node
        x.colour = node.colour
        node.colour = self.R
        return x
    
    def rightRotate(self, node):
        x = node.left
        node.left = x.right
        x.right = node
        x.colour = node.colour
        node.colour = self.R
        return x
    
    def flipColour(self, node):
        node.colour = not node.colour
        node.left.colour = self.B
        node.right.colour = self.B


In [16]:
rbt = RBTree()
rbt.insertElement('b')
rbt.insertElement('a')
rbt.insertElement('c')

print(rbt.searchElement('b'))
print(rbt.searchElement('a'))
print(rbt.searchElement('c'))

True
True
True


### Bloom Filter

In [17]:
# import mmh3
from bitarray import bitarray
  
class BloomFilter(object):
    def __init__(self, ItemNum, FPProb):
        # ItemNum : int, Number of items expected to be stored in bloom filter
        # FPProb : float, False Positive probability in decimal
        self.FPProb = FPProb
        self.size = self.getSize(ItemNum, FPProb)
        # number of hash functions to use
        self.hashNum = self.getHashNum(self.size, ItemNum)
        # Bit array of given size, and initialize all bits as 0
        self.bitArray = bitarray(self.size)
        self.bitArray.setall(0)

    def insertElement(self, item):
        digests = []
        for i in range(self.hashNum):
            # digest = self.mmh3Hash(item, i) % self.size
            digest = self.myHash(item, i) % self.size
            digests.append(digest)
            self.bitArray[digest] = True

    def searchElement(self, item):
        for i in range(self.hashNum):
            # digest = self.mmh3Hash(item, i) % self.size
            digest = self.myHash(item, i) % self.size
            if self.bitArray[digest] == False:
                return False
        return True

    def getSize(self, n, p):
        # Return the size of bit array: m = -(n * lg(p)) / (lg(2)^2)
        # n : number of items expected to be stored in filter
        # p : False Positive probability in decimal
        # m = -(n * math.log(p))/(math.log(2)**2)
        m = -(n * self.myLog(p)) / (self.myLog(2) ** 2)
        return int(m)

    def getHashNum(self, m, n):
        # Return number of hash functions: k = (m/n) * lg(2)
        # k = (m/n) * math.log(2)
        k = (m / n) * self.myLog(2)
        return int(k)
    
    def myLog(self, n):
        if n <= 0:
            raise ValueError("logarithm is undefined for non-positive values")
        if n == 1:
            return 0
        if n < 1:
            # log(xn) = -log(1/n) for n < 1
            return -self.myLog(1 / n)
        # Use the taylor series approximation for log(n) where |n-1| < 1
        taylor = (n - 1) / float(n + 1)
        term = taylor
        total = taylor
        num = 1
        while abs(term) > 1e-15:
            num += 2
            term *= taylor * taylor / float(num * (num - 1))
            total += term
        return 2 * total
    
    def mmh3Hash(self, key, seed):
        """
        Compute the 32-bit MurmurHash3 hash value for the given key and seed.
        """
        c1 = 0xcc9e2d51
        c2 = 0x1b873593
        r1 = 15
        r2 = 13
        m = 5
        n = 0xe6546b64
        length = len(key)

        h1 = seed
        rounded_end = (length & 0xfffffffc)
        for i in range(0, rounded_end, 4):
            k = (ord(key[i]) & 0xff) | ((ord(key[i+1]) & 0xff) << 8) | \
                ((ord(key[i+2]) & 0xff) << 16) | ((ord(key[i+3]) & 0xff) << 24)
            k = (k * c1) & 0xffffffff
            k = ((k << r1) | (k >> (32 - r1)))
            k = (k * c2) & 0xffffffff
            h1 = ((h1 ^ k) << r2) | (h1 >> (32 - r2))
            h1 = (h1 * m + n) & 0xffffffff

        if length & 0x03 == 3:
            k = (ord(key[rounded_end + 2]) & 0xff) << 16
            k |= (ord(key[rounded_end + 1]) & 0xff) << 8
            k |= ord(key[rounded_end]) & 0xff
            k = (k * c1) & 0xffffffff
            k = ((k << r1) | (k >> (32 - r1)))
            k = (k * c2) & 0xffffffff
            h1 = ((h1 ^ k) << r2) | (h1 >> (32 - r2))
            h1 = (h1 * m + n) & 0xffffffff

        elif length & 0x03 == 2:
            k = (ord(key[rounded_end + 1]) & 0xff) << 8
            k |= ord(key[rounded_end]) & 0xff
            k = (k * c1) & 0xffffffff
            k = ((k << r1) | (k >> (32 - r1)))
            k = (k * c2) & 0xffffffff
            h1 = ((h1 ^ k) << r2) | (h1 >> (32 - r2))
            h1 = (h1 * m + n) & 0xffffffff

        elif length & 0x03 == 1:
            k = ord(key[rounded_end]) & 0xff
            k = (k * c1) & 0xffffffff
            k = ((k << r1) | (k >> (32 - r1)))
            k = (k * c2) & 0xffffffff
            h1 = ((h1 ^ k) << r2) | (h1 >> (32 - r2))
            h1 = (h1 * m + n) & 0xffffffff

        h1 ^= length
        h1 = self.fmix(h1)
        return h1

    def fmix(self, h):
        """
        Apply the final mixing step of the MurmurHash3 algorithm.
        """
        h ^= h >> 16
        h = (h * 0x85ebca6b) & 0xffffffff
        h ^= h >> 13
        h = (h * 0xc2b2ae35) & 0xffffffff
        h ^= h >> 16
        return h
    
    def myHash(self, value, seed):
        hash_value = 0
        for char in value:
            hash_value += ord(char)
            hash_value += (hash_value << 10)
            hash_value ^= (hash_value >> 6)
        hash_value += (hash_value << 3)
        hash_value ^= (hash_value >> 11)
        hash_value += (hash_value << 15)
        return (hash_value ^ seed)

In [18]:
bf = BloomFilter(1000000, 0.01)

bf.insertElement('hello')
bf.insertElement('world')
bf.insertElement('foo')
bf.insertElement('bar')

print(bf.searchElement('hello'))
print(bf.searchElement('baz'))

True
False


Time Complexity: 
- Creating the Bloom filter is O(kn)
- Insert and Search an element: O(k)
- k is the number of hash functions and n is the number of elements

Space Complexity: 
- O(kb), where k is the number of hash functions and b is the bits number

## 2. Experimentally Evaluation

### Using Real Data

In [19]:
with open("./data/testfiles/test1-mobydick.txt") as f:
    for line in f:
        words = line.strip().split()
# words        

In [20]:
len(words)

209329

In [21]:
with open("./data/testfiles/test2-warpeace.txt") as f2:
    for line in f2:
        words2 = line.strip().split()
# words        

In [22]:
len(words2)

564236

In [23]:
with open("./data/testfiles/test3-dickens.txt") as f3:
    for line in f3:
        words3 = line.strip().split()
# words        

In [24]:
len(words3)

5149661

In [25]:
wordsforsearch = []
with open("./data/testfiles/test-search.txt") as fsearch:
    for line in fsearch:
        wordsforsearch.append(line.strip())
# wordsforsearch  

In [26]:
len(wordsforsearch)

544

#### The total amount of time of all insert operations

In [27]:
# timeit magic method: %timeit -r 100 -n 10 'operation'

In [28]:
import timeit

In [42]:
'''
sequential search
Create your class object here(for insert and search operations later)
'''
seqS = SequentialSearch()


In [ ]:
'''
binary search tree
Create your class object here(for insert and search operations later)
'''

In [ ]:
'''
balanced search tree
Create your class object here(for insert and search operations later)
'''
# Red Black Search Tree
rbt = RBTree()

In [38]:
BF = BloomFilter(len(words), 0.01)

Sequential Search

In [43]:
a = './data/testfiles/test1-mobydick.txt'
b = './data/testfiles/test2-warpeace.txt'
c = './data/testfiles/test3-dickens.txt'
insertFiles = [a,b,c]

for file in insertFiles:
    seqInsertStart = timeit.default_timer()
    seqS.insertElement(file)
    seqInsertEnd = timeit.default_timer()
    seqInsertTime = seqInsertEnd - seqInsertStart
    print(seqInsertTime)

0.028421400000070207
0.08031909999999698
0.8202785999999378


Binary Search Tree

In [ ]:
binaryInsertStart = timeit.default_timer()

'''
for word in words:
    Opearions
'''

binaryInsertEnd = timeit.default_timer()
binaryInsertTime = binaryInsertEnd - binaryInsertStart
binaryInsertTime

Balanced Search Tree

In [44]:
a = './data/testfiles/test1-mobydick.txt'
b = './data/testfiles/test2-warpeace.txt'
c = './data/testfiles/test3-dickens.txt'
insertFiles = [a,b,c]

for file in insertFiles:
    BSTInsertStart = timeit.default_timer()
    rbt.insertElement(file)
    BSTInsertEnd = timeit.default_timer()
    BSTInsertTime = BSTInsertEnd - BSTInsertStart
    print(BSTInsertTime)


1.3200000012147939e-05
1.6899999991437653e-05
4.79999994240643e-06


Bloom Filter

In [ ]:
BFInsertStart = timeit.default_timer()

for word in words:
    BF.insertElement(word)

BFInsertEnd = timeit.default_timer()
BFInsertTime = BFInsertEnd - BFInsertStart
BFInsertTime

1.275346599999466

#### The total amount of time of search operations

Sequential Search

In [45]:
a = './data/testfiles/test1-mobydick.txt'
b = './data/testfiles/test2-warpeace.txt'
c = './data/testfiles/test3-dickens.txt'
d = "./data/testfiles/test-search.txt"
insertFiles = [a,b,c]
for file in insertFiles:
    seqS.insertElement(file)
    SeqSearchStart = timeit.default_timer()
    seqS.searchElement(d)
    SeqSearchEnd = timeit.default_timer()
    SeqSearchTime = SeqSearchEnd - SeqSearchStart
    print(SeqSearchTime)

False ,['zero'] is not found
94.96285220000027
False ,['zero'] is not found
106.49899989999994
False ,['zero'] is not found
183.5646872999996


Binary Search Tree

In [ ]:
BinarySearchStart = timeit.default_timer()

# for word in wordsforsearch:
#     BF.searchElement(word)

BinarySearchEnd = timeit.default_timer()
BinarySearchTime = BinarySearchEnd - BinarySearchStart
BinarySearchTime

Balanced Search Tree

In [46]:
a = './data/testfiles/test1-mobydick.txt'
b = './data/testfiles/test2-warpeace.txt'
c = './data/testfiles/test3-dickens.txt'
d = "./data/testfiles/test-search.txt"
insertFiles = [a,b,c]

for file in insertFiles:
    rbt.insertElement(file)
    BSTSearchStart = timeit.default_timer()
    rbt.searchElement(d)
    BSTSearchEnd = timeit.default_timer()
    BSTSearchTime = BSTSearchEnd - BSTSearchStart
    print(BSTSearchTime)

2.4000000848900527e-06
8.000001798791345e-07
7.000003279244993e-07


Bloom Filter

In [ ]:
BFSearchStart = timeit.default_timer()

for word in wordsforsearch:
    BF.searchElement(word)

BFSearchEnd = timeit.default_timer()
BFSearchTime = BFSearchEnd - BFSearchStart
BFSearchTime

0.004203199998300988

### Using Synthetic Data

### different conditions:
- all values are the same
- all values are different
- values with high repetition rate
- values with low repetition rate
- adding numbers and symbols in each string
- all values are number (int/float/double/positive/negative/small/big)
- all values are list (list of string/num/list/set/dictionary/object/date)
- values with different data type mixed together
- ......

# --------

In [ ]:
# self

In [ ]:
import timeit
s1 = timeit.default_timer()

testBF = BloomFilter(len(words), 0.01)

e1 = timeit.default_timer()
print(e1 - s1)

8.689999958733097e-05


In [ ]:
s2 = timeit.default_timer()

for word in words:
    testBF.insert(word)

e2 = timeit.default_timer()
print(e2 - s2)

1.2846599000004062


In [ ]:
s3 = timeit.default_timer()

print(testBF.search("we"))

e3 = timeit.default_timer()
print(e3 - s3)

True
0.00012860000060754828


In [ ]:
%timeit -r 100 -n 10 testBF.search("abc")

2.02 µs ± 809 ns per loop (mean ± std. dev. of 100 runs, 10 loops each)
